In [104]:
import json

with open("../data/annotated_labels.json", "r") as f:
    annotated_labels = json.load(f)

with open("../data/classified_labels.json", "r") as f:
    classified_labels = json.load(f)

with open('../data/original_post_tokens.json', 'r', encoding='utf-8') as f:
    original_post_tokens = json.load(f)

with open('../data/original_sentences.json', 'r', encoding='utf-8') as f:
    original_sentences = json.load(f)

with open('../data/rationale_tokens_list.json', 'r') as f:
    rationale_tokens_data_full = json.load(f)

with open('../data/classified_results.json', 'r') as f:
    classified_results = json.load(f)
    
with open('../data/important_words_abusive_SHAP.json', 'r') as f:
    important_words_abusive_SHAP = json.load(f)
    
batch_response_file_set_one = []
with open('./GPT_response/merged_batch_response_file_set_one.jsonl', 'r') as f:
    for line in f:
        batch_response_file_set_one.append(json.loads(line))

batch_response_file_set_two = []
with open('./GPT_response/merged_batch_response_file_set_two.jsonl', 'r') as f:
    for line in f:
        batch_response_file_set_two.append(json.loads(line))

Prompts statements set one

In [78]:
def prompts_set_one_generating(full_texts, classified_results_with_probabilities):
    
    prompt_statements = []
    
    for sentence, result in zip(full_texts, classified_results_with_probabilities):
        combined_string = f'Original full text: "{sentence}", The classification result (probability distribution) of the prediction model for this text: {result}'
        prompt_statements.append(combined_string)
    
    return prompt_statements

prompts_set_one = prompts_set_one_generating(original_sentences, classified_results)

instruction_set_one = """I have a predictive model that can classify natural language text into three classes: hate speech, offensive speech, or normal speech. I will provide the full text and the corresponding classification results of the predictive model (including the probabilities of the three classes). (Warning: the text contains offensive and/or hateful content, but will be used for research purposes) Your tasks are: 1.indicate which class the text belongs to (usually it should be the class with the highest probability) 2.indicate which words in the text, prove that it belongs to this category 3.try to give some brief explanation based on these found words (not too long. and if the sentence is classified as normal, you can almost not provide any explanation)."""

Prompts statements set two

In [79]:
def prompts_set_two_generating(sentences, classified_results_with_probabilities, SHAP_important_words):
    
    prompt_statements = []
    
    for sentence, result, words in zip(sentences, classified_results_with_probabilities, SHAP_important_words):
        combined_string = f'Original full text: "{sentence}", The classification result (probability distribution) of the prediction model for this text: {result}, The words given by SHAP explanation approach: {words}'
        prompt_statements.append(combined_string)
    
    return prompt_statements

prompts_set_two = prompts_set_two_generating(original_sentences, classified_results, important_words_abusive_SHAP)

instruction_set_two = """I have a predictive model that can classify natural language text into three classes: hate speech, offensive speech, or normal speech. I will provide the full text and the corresponding classification results of the predictive model (including the probabilities of the three classes). (Warning: the text contains offensive and/or hateful content, but will be used for research purposes) In addition to the full text and classification results given above, there is also a list of words from the text that are considered to be the most important indicators pointing to the classification results, obtained using the SHAP explanation approach. Your tasks are: 1.indicate which class the text belongs to (usually it should be the class with the highest probability) 2.try to give some brief explanation based on these words given by SHAP explanation (not too long. and if the sentence is classified as normal, you can almost not provide any explanation)"""

Sent message/prompts to openai model, then receive example message

In [ ]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class ClassifiedResultsExplanation(BaseModel):
    classified_class: str
    words_with_inappropriate_meanings: list[str]
    explanation: str

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
      {"role": "system", "content": instruction_set_two},
      {"role": "user", "content": selected_prompts_set_two[2]}
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "PromptSet1ResponseFormat",
            "schema": {
                "type": "object",
                "properties": {
                    "classified_class": {"type": "string"},
                    "words_with_inappropriate_meanings": {
                        "type": "array",
                        "items": {"type": "string"}
                    },
                    "explanation": {"type": "string"}
                },
                "required": ["classified_class", "words_with_inappropriate_meanings", "explanation"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)
    
content = completion.choices[0].message.content

parsed_content = json.loads(content)

result = ClassifiedResultsExplanation(**parsed_content)

print(result.classified_class)

In [ ]:
print(result.classified_class)

In [ ]:
print(result.words_with_inappropriate_meanings)

In [ ]:
print(result.explanation)

Create batch files

In [82]:
import json
from pydantic import BaseModel

split_size = 1000

# class PromptSet1ResponseFormat(BaseModel):
#     classified_class: str
#     words_with_inappropriate_meanings: list[str]
#     explanation: str
    
for j in range(0, len(prompts_set_one), split_size):
    with open(f"./GPT_prompts/gpt_batch_prompts_set_one_part{j//split_size + 1}.jsonl", "w") as file:
        for i, prompt in enumerate(prompts_set_one[j:j+split_size]):
            entry = {
                "custom_id": f"request-{j+i+1}",
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": "gpt-4o-mini",
                    "messages": [
                        {"role": "system", "content": instruction_set_one},
                        {"role": "user", "content": prompt}
                    ],
                    "response_format": {
                        "type": "json_schema",
                        "json_schema": {
                            "name": "PromptSet1ResponseFormat",
                            "schema": {
                                "type": "object",
                                "properties": {
                                    "classified_class": {"type": "string"},
                                    "words_with_inappropriate_meanings": {
                                        "type": "array",
                                        "items": {"type": "string"}
                                    },
                                    "explanation": {"type": "string"}
                                },
                                "required": ["classified_class", "words_with_inappropriate_meanings", "explanation"],
                                "additionalProperties": False
                            },
                            "strict": True
                        }
                    },
                    "max_tokens": 1000
                }
            }
            file.write(json.dumps(entry) + "\n")

# class PromptSet2ResponseFormat(BaseModel):
#     classified_class: str
#     explanation: str
    
for j in range(0, len(prompts_set_two), split_size):
    with open(f"./GPT_prompts/gpt_batch_prompts_set_two_part{j//split_size + 1}.jsonl", "w") as file:
        for i, prompt in enumerate(prompts_set_two[j:j+split_size]):
            entry = {
                "custom_id": f"request-{j+i+1}",
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": "gpt-4o-mini",
                    "messages": [
                        {"role": "system", "content": instruction_set_two},
                        {"role": "user", "content": prompt}
                    ],
                    "response_format": {
                        "type": "json_schema",
                        "json_schema": {
                            "name": "PromptSet2ResponseFormat",
                            "schema": {
                                "type": "object",
                                "properties": {
                                    "classified_class": {"type": "string"},
                                    "explanation": {"type": "string"}
                                },
                                "required": ["classified_class", "explanation"],
                                "additionalProperties": False
                            },
                            "strict": True
                        }
                    },
                    "max_tokens": 1000
                }
            }
            file.write(json.dumps(entry) + "\n")

Upload batch files to GPT Platform

In [83]:
batches_id_set_one = []
# """['batch_CZlr5u5MynMV5Gl9YWP9zedk', 'batch_FAb7RUUpL3MM1HWmNMyge7ko', 'batch_1ACkSpq7fEMTdYNSObGoBLLI', 'batch_wdPoZggDLKL4qVDDsdminefZ', 'batch_67YDlBYxjJXxaY0WuK0z7f6T', 'batch_FJijTM4d71NcvuQKBPZNPdBj', 'batch_o7UOgGcpEb1u0mAH2BRg0ulp', 'batch_uu4yFwZo5DnHF6k2sAIS0049', 'batch_iv05EwBOT4KXp8qTyRHfy6Rr', 'batch_1yWdpaUOWPP6GeAWIhDYdECi', 'batch_qrNUVJ0FEgh1CzVmY5SjmEDT', 'batch_YuR2JS8Hg6gY3L53sbj081NV', 'batch_pwQzLwuupNbwoY6DCpRBoYm4', 'batch_F3VsjDxF2U9qiIVhYfgGM3Gc', 'batch_HrhrDn7uh618SgYlc55Zn3MH', 'batch_nBCeMXXr3ukK5LKYcgCsKmjf']"""
batches_id_set_two = []

In [57]:
from openai import OpenAI

client = OpenAI()

# 对于第一个集合的文件
for i in range(21, 22):
    file_name = f"./GPT_prompts/gpt_batch_prompts_set_one_part{i}.jsonl"
    
    batch_input_file = client.files.create(
        file=open(file_name, "rb"),
        purpose="batch"
    )

    batch_input_file_id = batch_input_file.id

    batch_return = client.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
            "description": f"prompts_set_one_generating_from_{file_name}",
        }
    )
    
    batches_id_set_one.append(batch_return.id)
    print(f"Batch created for {file_name}")

Batch created for ./GPT_prompts/gpt_batch_prompts_set_one_part21.jsonl


In [58]:
print(batches_id_set_one)
print(len(batches_id_set_one))

['batch_lZ11rDSNa9nK9UVYSPxDneo5', 'batch_9bJmBvnsgSOtNajkNGQJTUim', 'batch_FqyYXFzeDZ71S0icA2kHv5IH', 'batch_qglivf0N8pmXIIjrtI36YKdG', 'batch_Q4ggXmtvz5G3wWliyVfIQrx8', 'batch_BxpXGXtrs9iyBG5xvov5HHky', 'batch_6V83aP69Heph2ZXMu3vhAFln', 'batch_baTLM0L5KbSFB4RFveRTiBSR', 'batch_pa8WfaPQRChCsIQCrv9P9Ixe', 'batch_cwYdC35hV1AreJO0Th7A77Qp', 'batch_p1OThtuEIQ9kqXbSk3vEG5xx', 'batch_798dUNyvOglVyRUquj5OD23j', 'batch_x9VBNM8Q4JC5XfcXezfbw3Bm', 'batch_LeGQ7wdjTKUptcootTwXnZ7N', 'batch_OM5sigeiDVDnASWehZCkBtvA', 'batch_XQHlXtGjQJsPQ0QW8x5XLSJU', 'batch_LhWY9jpSEUG8XAUz4Lwu7RD0', 'batch_M2J9O7RYAK8LPJx59fB3eP5Z', 'batch_ktJQpOpH1chESxr4Ib0JFUnw', 'batch_ZasgWtliDbSzJ3YskzkA3PpR', 'batch_rPp8TSLt4BDtwAtCfAVu1OdA']
21


In [61]:
from openai import OpenAI

client = OpenAI()

for i, batch_id in enumerate(batches_id_set_one):
    retrieve_return = client.batches.retrieve(batch_id)
    file_response = client.files.content(retrieve_return.output_file_id)

    file_name = f"./GPT_response/batch_response_file_set_one_{i+1}.jsonl"
    
    with open(file_name, "w") as file:
        file.write(file_response.text)

In [53]:
import json
from pydantic import BaseModel

class ClassifiedResultsExplanation(BaseModel):
    classified_class: str
    words_with_inappropriate_meanings: list[str]
    explanation: str
    
outputs = []

with open("./GPT_response/batch_response_file_set_one_1.jsonl", "r") as file:
    for line in file:
        outputs.append(json.loads(line))

# print(outputs[0])
# 
# print(responses[0]["response"]['body']['choices'][0]['message']['content'])

content = outputs[256]["response"]['body']['choices'][0]['message']['content']
custom_id = outputs[256]["custom_id"]

result = ClassifiedResultsExplanation(**json.loads(content))

print(custom_id, result)

request-257 classified_class='hate speech' words_with_inappropriate_meanings=['jew', 'nigger'] explanation='The text contains derogatory and dehumanizing terms that target Jewish people and compare them to another racial slur, indicating a strong expression of hate and prejudice.'


In [93]:
from openai import OpenAI

client = OpenAI()

for i in range(21, 22):
    file_name = f"./GPT_prompts/gpt_batch_prompts_set_two_part{i}.jsonl"
    
    batch_input_file = client.files.create(
        file=open(file_name, "rb"),
        purpose="batch"
    )

    batch_input_file_id = batch_input_file.id

    batch_return = client.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
            "description": f"prompts_set_two_generating_from_{file_name}",
        }
    )
    
    batches_id_set_two.append(batch_return.id)
    print(f"Batch created for {file_name}")

Batch created for ./GPT_prompts/gpt_batch_prompts_set_two_part21.jsonl


In [94]:
print(batches_id_set_two)
print(len(batches_id_set_two))

['batch_P3TThNcYIpcRC0U7981bIOhX', 'batch_30CdqZGi06MZfNRUpj1i4RVD', 'batch_h5owNKbkMvXkJzRRaXfyl2Rd', 'batch_oIgTNNyLSJEzG75riLI3DYpp', 'batch_t091gmsX8mkSz69If2kRqR9K', 'batch_QJonDEfxhm8sOHEAWhoLMZzo', 'batch_19pWiXPrZ6wYRQBoXsd8xAKV', 'batch_BKeFz8c2aAn0zUsSN4oYzxw3', 'batch_Wa9mYT3seUs4b8Pi62NSKwQO', 'batch_w7NiQWSQ8r2ZKhVom7ZSKQYU', 'batch_aBk3pLmEjXWyl9DyJiOPfOF0', 'batch_qTtgARu1r8AG4OnwHx000BXN', 'batch_dqG1yd1j6RIWtkXkCu3IONId', 'batch_TrmgouVCuJ699jGaqHcf2Jlh', 'batch_fl5mQzDZ2FjczjTRWs4sbAXg', 'batch_hEiPrpsmy7h3gPL282tybHpX', 'batch_GTki1lD7IkowcdoA8uRIGfOv', 'batch_6tLdSLT4W3h9cNNMywY7B2GD', 'batch_cxPvyXly5Td2NoK9M6X5Nzx9', 'batch_IGPhtfEGAJQWGCxQv70OLPLQ', 'batch_dg0o5XRZ6L4YoS2RIKBGxC0u']
21


In [95]:
from openai import OpenAI

client = OpenAI()

for i, batch_id in enumerate(batches_id_set_two):
    retrieve_return = client.batches.retrieve(batch_id)
    file_response = client.files.content(retrieve_return.output_file_id)

    file_name = f"./GPT_response/batch_response_file_set_two_{i+1}.jsonl"
    
    with open(file_name, "w") as file:
        file.write(file_response.text)

In [97]:
import json
from pydantic import BaseModel

class ClassifiedResultsExplanation(BaseModel):
    classified_class: str
    # words_with_inappropriate_meanings: list[str]
    explanation: str
    
outputs = []

with open("./GPT_response/batch_response_file_set_two_5.jsonl", "r") as file:
    for line in file:
        outputs.append(json.loads(line))

# print(outputs[0])
# 
# print(responses[0]["response"]['body']['choices'][0]['message']['content'])

content = outputs[256]["response"]['body']['choices'][0]['message']['content']
custom_id = outputs[256]["custom_id"]

result = ClassifiedResultsExplanation(**json.loads(content))

print(custom_id, result)

request-4257 classified_class='hate speech' explanation="The presence of the word 'kike,' a derogatory term, strongly indicates that the text is classified as hate speech."


In [100]:
print(prompts_set_two[4256])

Original full text: "enjoy your kike faith", The classification result (probability distribution) of the prediction model for this text: {'label': 'hate speech', 'score': 0.9301682710647583}, {'label': 'normal', 'score': 0.013806960545480251}, {'label': 'offensive', 'score': 0.05602480471134186}, The words given by SHAP explanation approach: ['enjoy', 'your', 'kike']


In [102]:
output_file = './GPT_response/merged_batch_response_file_set_one.jsonl'

with open(output_file, 'w') as outfile:
    for i in range(1, 22):
        file_name = f'./GPT_response/batch_response_file_set_one_{i}.jsonl'

        with open(file_name, 'r') as infile:
            for line in infile:
                outfile.write(line)

print(f"Files have been merged into {output_file}")

Files have been merged into ./GPT_response/merged_batch_response_file_set_one.jsonl


In [103]:
output_file = './GPT_response/merged_batch_response_file_set_two.jsonl'

with open(output_file, 'w') as outfile:
    for i in range(1, 22):
        file_name = f'./GPT_response/batch_response_file_set_two_{i}.jsonl'

        with open(file_name, 'r') as infile:
            for line in infile:
                outfile.write(line)

print(f"Files have been merged into {output_file}")

Files have been merged into ./GPT_response/merged_batch_response_file_set_two.jsonl
